# Animations

This is an example of making animations by updating the plot.
First, create a simple line, a tracker and a particle set, and track it for a couple turns:

In [1]:
import xtrack as xt
import xpart as xp
import xplt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

np.random.seed(43543557)

In [ ]:
## Generate a simple 6-fold symmetric FODO lattice
n = 6
elements = []
for i in range(n):
    elements.extend(
        [
            xt.Drift(length=0.7),
            xt.Multipole(length=0.3, knl=[0, +0.63], ksl=[0, 0]),
            xt.Drift(length=0.7),
            xt.Multipole(length=0.5, knl=[np.pi / n], hxl=[np.pi / n]),
            xt.Drift(length=0.4),
            xt.Multipole(length=0.2, knl=[0, 0, 0.5 * np.sin(2 * np.pi * (i / n))]),
            xt.Drift(length=0.3),
            xt.Multipole(length=0.3, knl=[0, -0.48], ksl=[0, 0]),  # -0.4
            xt.Drift(length=0.7),
            xt.Multipole(length=0.5, knl=[np.pi / n], hxl=[np.pi / n]),
            xt.Drift(length=2.2),
        ][:: -1 if i % 2 else 1]
    )

line = xt.Line(elements=elements)

## Create the tracker
tracker = xt.Tracker(line=line)

In [3]:
## Generate particles
line.particle_ref = xp.Particles(mass0=xp.PROTON_MASS_EV, q0=1, p0c=1e9)
nparticles = int(1e3)

# Transverse distribution (gaussian)
norm_emitt_x = 2e-6  # normalized 1-sigma emittance in m*rad (=beta*gamma*emitt_x)
norm_emitt_y = 1e-6  # normalized 1-sigma emittance in m*rad (=beta*gamma*emitt_y)
x, px = xp.generate_2D_gaussian(nparticles)
y, py = xp.generate_2D_gaussian(nparticles)

# Longitudinal distribution (coasting beam)
rel_momentum_spread = 1e-4  # relative momentum spread ( P/p0 - 1 )
zeta = line.get_length() * np.random.uniform(-0.5, 0.5, nparticles)
delta = rel_momentum_spread * xp.generate_2D_gaussian(nparticles)[0]

particles = xp.build_particles(
    tracker=tracker,
    particle_ref=line.particle_ref,
    x_norm=x,
    px_norm=px,
    nemitt_x=norm_emitt_x,
    y_norm=y,
    py_norm=py,
    nemitt_y=norm_emitt_y,
    method="4d",  # for twiss (default is 6d, won't work without a cavity)
    zeta=zeta,
    delta=delta,
)

In [4]:
## Generate tracking data
tracker.track(particles, num_turns=50, turn_by_turn_monitor=True)

Adjust some parameters to improve rendering performance of the animation:

In [5]:
# use a faster backend
xplt.mpl.rcParams["backend"] = "nbagg"
# increase buffer size
xplt.mpl.rcParams["animation.embed_limit"] = 100  # MB
# use a faster style
xplt.mpl.style.use("fast")

## Phase space animation

Use a `FuncAnimation` together with the `update` method to create an animation:

In [6]:
plot = xplt.PhaseSpacePlot(animated=True, std=True, mean=True)
particle_data = tracker.record_last_track


def animate(i):
    turn = particle_data.at_turn[0, i]
    plot.fig.suptitle(f"Turn {turn}")
    return plot.update(
        particle_data,
        mask=(slice(None), i),  # select all particles and a single turn
        autoscale=(i == 0),
    )


anim = FuncAnimation(plot.fig, animate, frames=range(0, 50, 3), interval=100, blit=True)

display(HTML(anim.to_jshtml()))
# anim.save('anim.gif', dpi=150, progress_callback=lambda i,n: print(f'\rSaving... {i*100/n:3.1f}% ({i}/{n})', end=''))

In [7]:
# especially for larger animations, it is good practice to clean up:
plot.fig.clear()
# xplt.plt.close()
del plot
import gc

gc.collect();